In [2]:
import cv2
import os
import numpy as np
from skimage import color,data,restoration
from math import log10,sqrt
from scipy import signal
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import mean_squared_error as mse
from scipy.ndimage.filters import median_filter
import pandas as pd

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path = '/content/drive/MyDrive/Image_Processing_Plants/dataset'

folders = []
for root,dirs,files in os.walk(path):
  for dir in dirs:
    folders.append(os.path.join(root,dir))

In [4]:
def PSNR(old,new):
  mse = np.mean((old-new)**2)
  if(mse == 0):
    return 100
  max_pixel = 255.0
  psnr = 20*log10(max_pixel/sqrt(mse))
  return psnr


In [5]:
def unsharp(image, sigma, strength):
  image_mf = median_filter(image, sigma)
  lap = cv2.Laplacian(image_mf,cv2.CV_64F)
  sharp = image-strength*lap
  sharp[sharp>255] = 255
  sharp[sharp<0] = 0    
  return sharp

In [6]:
def weiner_filter(image1):
  psf = np.ones((3,3),np.float32)/9
  new_image = restoration.wiener(image1,psf,1100)
  return new_image


In [7]:
from numba import njit,prange

# Adaptive Median Filter
@njit
def padding(img,pad):
    padded_img = np.zeros((img.shape[0]+2*pad,img.shape[1]+2*pad))
    padded_img[pad:-pad,pad:-pad] = img
    return padded_img

@njit(parallel=True)
def AdaptiveMedianFilter(img,s=3,sMax=7):
    if len(img.shape) == 3:
        raise Exception ("Single channel image only")

    H,W = img.shape
    a = sMax//2
    padded_img = padding(img,a)

    f_img = np.zeros(padded_img.shape)

    for i in prange(a,H+a+1):
        for j in range(a,W+a+1):
            value = Lvl_A(padded_img,i,j,s,sMax)
            f_img[i,j] = value

    return f_img[a:-a,a:-a] 

@njit
def Lvl_A(mat,x,y,s,sMax):
    window = mat[x-(s//2):x+(s//2)+1,y-(s//2):y+(s//2)+1]
    Zmin = np.min(window)
    Zmed = np.median(window)
    Zmax = np.max(window)

    A1 = Zmed - Zmin
    A2 = Zmed - Zmax

    if A1 > 0 and A2 < 0:
        return Lvl_B(window, Zmin, Zmed, Zmax)
    else:
        s += 2 
        if s <= sMax:
            return Lvl_A(mat,x,y,s,sMax)
        else:
             return Zmed

@njit
def Lvl_B(window, Zmin, Zmed, Zmax):
    h,w = window.shape

    Zxy = window[h//2,w//2]
    B1 = Zxy - Zmin
    B2 = Zxy - Zmax

    if B1 > 0 and B2 < 0 :
        return Zxy
    else:
        return Zmed

In [8]:
from scipy.ndimage import maximum_filter, minimum_filter

def midpoint(img):
    maxf = maximum_filter(img, (3, 3))
    minf = minimum_filter(img, (3, 3))
    midpoint = (maxf + minf) / 2
    return midpoint

In [9]:
def contraharmonic_mean(img, size, Q):
    num = np.power(img, Q + 1)
    denom = np.power(img, Q)
    kernel = np.full(size, 1.0)
    result = cv2.filter2D(num, -1, kernel) / cv2.filter2D(denom, -1, kernel)
    return result


In [10]:
# Gabor Filter
def Gabor_filter(K_size=111, Sigma=10, Gamma=1.2, Lambda=10, Psi=0, angle=0):
	d = K_size // 2
	gabor = np.zeros((K_size, K_size), dtype=np.float32)
	for y in range(K_size):
		for x in range(K_size):
			px = x - d
			py = y - d
			theta = angle / 180. * np.pi
			_x = np.cos(theta) * px + np.sin(theta) * py
			_y = -np.sin(theta) * px + np.cos(theta) * py
			gabor[y, x] = np.exp(-(_x**2 + Gamma**2 * _y**2) / (2 * Sigma**2)) * np.cos(2*np.pi*_x/Lambda + Psi)
	gabor /= np.sum(np.abs(gabor))
	return gabor



def Gabor_filtering(gray, K_size=111, Sigma=10, Gamma=1.2, Lambda=10, Psi=0, angle=0):
    H, W = gray.shape
    gray = np.pad(gray, (K_size//2, K_size//2), 'edge')
    out = np.zeros((H, W), dtype=np.float32)
    gabor = Gabor_filter(K_size=K_size, Sigma=Sigma, Gamma=Gamma, Lambda=Lambda, Psi=0, angle=angle)
    for y in range(H):
        for x in range(W):
            out[y, x] = np.sum(gray[y : y + K_size, x : x + K_size] * gabor)
    out = np.clip(out, 0, 255)
    out = out.astype(np.uint8)
    return out


def Gabor_process(img):
    H, W = img.shape
    gray = img
    As = [0,30,60,90,120,150]
    out = np.zeros([H, W], dtype=np.float32)
    for i, A in enumerate(As):
        _out = Gabor_filtering(gray, K_size=11, Sigma=1.5, Gamma=1.2, Lambda=3, angle=A)
        out += _out
    out = out / out.max() * 255
    out = out.astype(np.uint8)
    return out


# Image Quality metrics files


In [11]:
import csv 
f1 = open("/content/drive/MyDrive/Image_Processing_Plants/Filter_Img_Metrics/PSNR_curl_clr.csv",'w')
writer1 = csv.writer(f1)

f11 = open("/content/drive/MyDrive/Image_Processing_Plants/Filter_Img_Metrics/PSNR_curl_gray.csv",'w')
writer11 = csv.writer(f11)

f2 = open("/content/drive/MyDrive/Image_Processing_Plants/Filter_Img_Metrics/PSNR_Pests_clr.csv",'w')
writer2 = csv.writer(f2)

f22 = open("/content/drive/MyDrive/Image_Processing_Plants/Filter_Img_Metrics/PSNR_Pests_gray.csv",'w')
writer22 = csv.writer(f22)

f3 = open("/content/drive/MyDrive/Image_Processing_Plants/Filter_Img_Metrics/PSNR_Healthy_clr.csv",'w')
writer3 = csv.writer(f3)

f33 = open("/content/drive/MyDrive/Image_Processing_Plants/Filter_Img_Metrics/PSNR_Healthy_gray.csv",'w')
writer33 = csv.writer(f33)

f4 = open("/content/drive/MyDrive/Image_Processing_Plants/Filter_Img_Metrics/PSNR_Fusarium_clr.csv",'w')
writer4 = csv.writer(f4)

f44 = open("/content/drive/MyDrive/Image_Processing_Plants/Filter_Img_Metrics/PSNR_Fusarium_gray.csv",'w')
writer44 = csv.writer(f44)

f5 = open("/content/drive/MyDrive/Image_Processing_Plants/Filter_Img_Metrics/PSNR_Bacterial_clr.csv",'w')
writer5 = csv.writer(f5)

f55 = open("/content/drive/MyDrive/Image_Processing_Plants/Filter_Img_Metrics/PSNR_Bacterial_gray.csv",'w')
writer55 = csv.writer(f55)


In [12]:
s1 = open("/content/drive/MyDrive/Image_Processing_Plants/Filter_Img_Metrics/SSIM_curl_clr.csv",'w')
swriter1 = csv.writer(s1)

s11 = open("/content/drive/MyDrive/Image_Processing_Plants/Filter_Img_Metrics/SSIM_curl_gray.csv",'w')
swriter11 = csv.writer(s11)

s2 = open("/content/drive/MyDrive/Image_Processing_Plants/Filter_Img_Metrics/SSIM_Pests_clr.csv",'w')
swriter2 = csv.writer(s2)

s22 = open("/content/drive/MyDrive/Image_Processing_Plants/Filter_Img_Metrics/SSIM_Pests_gray.csv",'w')
swriter22 = csv.writer(s22)

s3 = open("/content/drive/MyDrive/Image_Processing_Plants/Filter_Img_Metrics/SSIM_Healthy_clr.csv",'w')
swriter3 = csv.writer(s3)

s33 = open("/content/drive/MyDrive/Image_Processing_Plants/Filter_Img_Metrics/SSIM_Healthy_gray.csv",'w')
swriter33 = csv.writer(s33)

s4 = open("/content/drive/MyDrive/Image_Processing_Plants/Filter_Img_Metrics/SSIM_Fusarium_clr.csv",'w')
swriter4 = csv.writer(s4)

s44 = open("/content/drive/MyDrive/Image_Processing_Plants/Filter_Img_Metrics/SSIM_Fusarium_gray.csv",'w')
swriter44 = csv.writer(s44)

s5 = open("/content/drive/MyDrive/Image_Processing_Plants/Filter_Img_Metrics/SSIM_Bacterial_clr.csv",'w')
swriter5 = csv.writer(s5)

s55 = open("/content/drive/MyDrive/Image_Processing_Plants/Filter_Img_Metrics/SSIM_Bacterial_gray.csv",'w')
swriter55 = csv.writer(s55)


In [13]:
m1 = open("/content/drive/MyDrive/Image_Processing_Plants/Filter_Img_Metrics/MSE_curl_clr.csv",'w')
mwriter1 = csv.writer(m1)

m11 = open("/content/drive/MyDrive/Image_Processing_Plants/Filter_Img_Metrics/MSE_curl_gray.csv",'w')
mwriter11 = csv.writer(m11)

m2 = open("/content/drive/MyDrive/Image_Processing_Plants/Filter_Img_Metrics/MSE_Pests_clr.csv",'w')
mwriter2 = csv.writer(m2)

m22 = open("/content/drive/MyDrive/Image_Processing_Plants/Filter_Img_Metrics/MSE_Pests_gray.csv",'w')
mwriter22 = csv.writer(m22)

m3 = open("/content/drive/MyDrive/Image_Processing_Plants/Filter_Img_Metrics/MSE_Healthy_clr.csv",'w')
mwriter3 = csv.writer(m3)

m33 = open("/content/drive/MyDrive/Image_Processing_Plants/Filter_Img_Metrics/MSE_Healthy_gray.csv",'w')
mwriter33 = csv.writer(m33)

m4 = open("/content/drive/MyDrive/Image_Processing_Plants/Filter_Img_Metrics/MSE_Fusarium_clr.csv",'w')
mwriter4 = csv.writer(m4)

m44 = open("/content/drive/MyDrive/Image_Processing_Plants/Filter_Img_Metrics/MSE_Fusarium_gray.csv",'w')
mwriter44 = csv.writer(m44)

m5 = open("/content/drive/MyDrive/Image_Processing_Plants/Filter_Img_Metrics/MSE_Bacterial_clr.csv",'w')
mwriter5 = csv.writer(m5)

m55 = open("/content/drive/MyDrive/Image_Processing_Plants/Filter_Img_Metrics/MSE_Bacterial_gray.csv",'w')
mwriter55 = csv.writer(m55)


# Filtering Techniques evaluation

In [ ]:
for dir in folders:
  for _,_,files in os.walk(dir):
    for f in files:

# curl data set      
      if(f.startswith("curl_")):
        image = cv2.imread(os.path.join(dir,f))
        # mean filter
        kernel = np.ones((3,3),np.float32)/9
        mean_image = cv2.filter2D(image,-1,kernel)
        psnr_mean = PSNR(image,mean_image)
        ssim_mean = ssim(image,mean_image,multichannel=True)
        mse_mean = mse(image,mean_image)

        # median filter
        median_image = cv2.medianBlur(image,9)
        psnr_median = PSNR(image,median_image)
        ssim_median = ssim(image,median_image,multichannel=True)
        mse_median = mse(image,median_image)
        
        # gaussian filter
        gaussian_image = cv2.GaussianBlur(image,(5,5),0)
        psnr_gaussian = PSNR(image,gaussian_image)
        ssim_gaussian = ssim(image,gaussian_image,multichannel=True)
        mse_gaussian = mse(image,gaussian_image)
        
        # unsharp mask filter
        sharp = np.zeros_like(image)
        for i in range(3):
          sharp[:,:,i] = unsharp(image[:,:,i], 5, 0.8)
        psnr_unsharp = PSNR(image,sharp)
        ssim_unsharp = ssim(image,sharp,multichannel=True)
        mse_unsharp = mse(image,sharp)
        
        # wiener filter
        wiener_image = np.ones_like(image)
        for i in range(3):
          wiener_image[:,:,i] = weiner_filter(wiener_image[:,:,i])
        psnr_wiener = PSNR(image,wiener_image)
        ssim_wiener = ssim(image,wiener_image,multichannel=True)
        mse_wiener = mse(image,wiener_image)

        # contraharmonic mean filter
        chmean_image = contraharmonic_mean(image,(3,3),0.5)
        psnr_chmean = PSNR(image,chmean_image)
        ssim_chmean = ssim(image,chmean_image,multichannel=True)
        mse_chmean = mse(image,chmean_image)

        # Arithmetic filter with sharpening kernel
        kernel_sharpening = np.array([[-1,-1,-1],
                              [-1, 9,-1],
                              [-1,-1,-1]])
        amsharp_image = cv2.filter2D(image, -1, kernel_sharpening)
        psnr_amsharp = PSNR(image,amsharp_image)
        ssim_amsharp = ssim(image,amsharp_image,multichannel=True)
        mse_amsharp = mse(image,amsharp_image)

        # gray scale images
        bwimage = cv2.imread(os.path.join(dir,f),cv2.IMREAD_GRAYSCALE)
        # Laplacian Filter 
        dest = cv2.Laplacian(bwimage, cv2.CV_16S, ksize=3)
        laplacian_image = cv2.convertScaleAbs(dest)
        psnr_laplacian = PSNR(bwimage,laplacian_image)
        ssim_laplacian = ssim(bwimage,laplacian_image)
        mse_laplacian = mse(bwimage,laplacian_image)

        # LoG Filter
        gaussian_image = cv2.GaussianBlur(bwimage,(5,5),0)
        dest = cv2.Laplacian(gaussian_image, cv2.CV_64F, ksize=3)
        log_image = cv2.convertScaleAbs(dest)
        psnr_log = PSNR(bwimage,log_image)
        ssim_log = ssim(bwimage,log_image)
        mse_log = mse(bwimage,log_image)
        
        # Adaptive Median Filter
        admed_image = AdaptiveMedianFilter(bwimage)
        psnr_admed = PSNR(bwimage,admed_image)
        ssim_admed = ssim(bwimage,admed_image)
        mse_admed = mse(bwimage,admed_image)
        
        # Midpoint Filter
        midpoint_image = midpoint(bwimage)
        psnr_midpoint = PSNR(bwimage,midpoint_image)
        ssim_midpoint = ssim(bwimage,midpoint_image)
        mse_midpoint = mse(bwimage,midpoint_image)
        
        # Gabor Filter
        gabor_image = Gabor_process(bwimage)
        psnr_gabor = PSNR(bwimage,gabor_image)
        ssim_gabor = ssim(bwimage,gabor_image)
        mse_gabor = mse(bwimage,gabor_image)
        
        print(writer1.writerow([psnr_mean,psnr_median,psnr_gaussian,psnr_unsharp,psnr_wiener,psnr_chmean,psnr_amsharp]))
        writer11.writerow([psnr_laplacian,psnr_log,psnr_admed,psnr_midpoint,psnr_gabor])
        swriter1.writerow([ssim_mean,ssim_median,ssim_gaussian,ssim_unsharp,ssim_wiener,ssim_chmean,ssim_amsharp])
        swriter11.writerow([ssim_laplacian,ssim_log,ssim_admed,ssim_midpoint,ssim_gabor])
        mwriter1.writerow([mse_mean,mse_median,mse_gaussian,mse_unsharp,mse_wiener,mse_chmean,mse_amsharp])
        mwriter11.writerow([mse_laplacian,mse_log,mse_admed,mse_midpoint,mse_gabor])

# Pests Data Set      
      elif(f.startswith("pests_")): 
        image = cv2.imread(os.path.join(dir,f))
        # mean filter
        kernel = np.ones((3,3),np.float32)/9
        mean_image = cv2.filter2D(image,-1,kernel)
        psnr_mean = PSNR(image,mean_image)
        ssim_mean = ssim(image,mean_image,multichannel=True)
        mse_mean = mse(image,mean_image)

        # median filter
        median_image = cv2.medianBlur(image,9)
        psnr_median = PSNR(image,median_image)
        ssim_median = ssim(image,median_image,multichannel=True)
        mse_median = mse(image,median_image)
        
        # gaussian filter
        gaussian_image = cv2.GaussianBlur(image,(5,5),0)
        psnr_gaussian = PSNR(image,gaussian_image)
        ssim_gaussian = ssim(image,gaussian_image,multichannel=True)
        mse_gaussian = mse(image,gaussian_image)
        
        # unsharp mask filter
        sharp = np.zeros_like(image)
        for i in range(3):
          sharp[:,:,i] = unsharp(image[:,:,i], 5, 0.8)
        psnr_unsharp = PSNR(image,sharp)
        ssim_unsharp = ssim(image,sharp,multichannel=True)
        mse_unsharp = mse(image,sharp)
        
        # wiener filter
        wiener_image = np.ones_like(image)
        for i in range(3):
          wiener_image[:,:,i] = weiner_filter(wiener_image[:,:,i])
        psnr_wiener = PSNR(image,wiener_image)
        ssim_wiener = ssim(image,wiener_image,multichannel=True)
        mse_wiener = mse(image,wiener_image)

        # contraharmonic mean filter
        chmean_image = contraharmonic_mean(image,(3,3),0.5)
        psnr_chmean = PSNR(image,chmean_image)
        ssim_chmean = ssim(image,chmean_image,multichannel=True)
        mse_chmean = mse(image,chmean_image)

        # Arithmetic filter with sharpening kernel
        kernel_sharpening = np.array([[-1,-1,-1],
                              [-1, 9,-1],
                              [-1,-1,-1]])
        amsharp_image = cv2.filter2D(image, -1, kernel_sharpening)
        psnr_amsharp = PSNR(image,amsharp_image)
        ssim_amsharp = ssim(image,amsharp_image,multichannel=True)
        mse_amsharp = mse(image,amsharp_image)

        # gray scale images
        bwimage = cv2.imread(os.path.join(dir,f),cv2.IMREAD_GRAYSCALE)
        # Laplacian Filter 
        dest = cv2.Laplacian(bwimage, cv2.CV_16S, ksize=3)
        laplacian_image = cv2.convertScaleAbs(dest)
        psnr_laplacian = PSNR(bwimage,laplacian_image)
        ssim_laplacian = ssim(bwimage,laplacian_image)
        mse_laplacian = mse(bwimage,laplacian_image)

        # LoG Filter
        gaussian_image = cv2.GaussianBlur(bwimage,(5,5),0)
        dest = cv2.Laplacian(gaussian_image, cv2.CV_64F, ksize=3)
        log_image = cv2.convertScaleAbs(dest)
        psnr_log = PSNR(bwimage,log_image)
        ssim_log = ssim(bwimage,log_image)
        mse_log = mse(bwimage,log_image)
        
        # Adaptive Median Filter
        admed_image = AdaptiveMedianFilter(bwimage)
        psnr_admed = PSNR(bwimage,admed_image)
        ssim_admed = ssim(bwimage,admed_image)
        mse_admed = mse(bwimage,admed_image)
        
        # Midpoint Filter
        midpoint_image = midpoint(bwimage)
        psnr_midpoint = PSNR(bwimage,midpoint_image)
        ssim_midpoint = ssim(bwimage,midpoint_image)
        mse_midpoint = mse(bwimage,midpoint_image)
        
        # Gabor Filter
        gabor_image = Gabor_process(bwimage)
        psnr_gabor = PSNR(bwimage,gabor_image)
        ssim_gabor = ssim(bwimage,gabor_image)
        mse_gabor = mse(bwimage,gabor_image)
        
        writer2.writerow([psnr_mean,psnr_median,psnr_gaussian,psnr_unsharp,psnr_wiener,psnr_chmean,psnr_amsharp])
        writer22.writerow([psnr_laplacian,psnr_log,psnr_admed,psnr_midpoint,psnr_gabor])
        swriter2.writerow([ssim_mean,ssim_median,ssim_gaussian,ssim_unsharp,ssim_wiener,ssim_chmean,ssim_amsharp])
        swriter22.writerow([ssim_laplacian,ssim_log,ssim_admed,ssim_midpoint,ssim_gabor])
        mwriter2.writerow([mse_mean,mse_median,mse_gaussian,mse_unsharp,mse_wiener,mse_chmean,mse_amsharp])
        mwriter22.writerow([mse_laplacian,mse_log,mse_admed,mse_midpoint,mse_gabor])

# Healthy Data set      
      elif(f.startswith("healthy")):
        image = cv2.imread(os.path.join(dir,f))
        # mean filter
        kernel = np.ones((3,3),np.float32)/9
        mean_image = cv2.filter2D(image,-1,kernel)
        psnr_mean = PSNR(image,mean_image)
        ssim_mean = ssim(image,mean_image,multichannel=True)
        mse_mean = mse(image,mean_image)

        # median filter
        median_image = cv2.medianBlur(image,9)
        psnr_median = PSNR(image,median_image)
        ssim_median = ssim(image,median_image,multichannel=True)
        mse_median = mse(image,median_image)
        
        # gaussian filter
        gaussian_image = cv2.GaussianBlur(image,(5,5),0)
        psnr_gaussian = PSNR(image,gaussian_image)
        ssim_gaussian = ssim(image,gaussian_image,multichannel=True)
        mse_gaussian = mse(image,gaussian_image)
        
        # unsharp mask filter
        sharp = np.zeros_like(image)
        for i in range(3):
          sharp[:,:,i] = unsharp(image[:,:,i], 5, 0.8)
        psnr_unsharp = PSNR(image,sharp)
        ssim_unsharp = ssim(image,sharp,multichannel=True)
        mse_unsharp = mse(image,sharp)
        
        # wiener filter
        wiener_image = np.ones_like(image)
        for i in range(3):
          wiener_image[:,:,i] = weiner_filter(wiener_image[:,:,i])
        psnr_wiener = PSNR(image,wiener_image)
        ssim_wiener = ssim(image,wiener_image,multichannel=True)
        mse_wiener = mse(image,wiener_image)

        # contraharmonic mean filter
        chmean_image = contraharmonic_mean(image,(3,3),0.5)
        psnr_chmean = PSNR(image,chmean_image)
        ssim_chmean = ssim(image,chmean_image,multichannel=True)
        mse_chmean = mse(image,chmean_image)

        # Arithmetic filter with sharpening kernel
        kernel_sharpening = np.array([[-1,-1,-1],
                              [-1, 9,-1],
                              [-1,-1,-1]])
        amsharp_image = cv2.filter2D(image, -1, kernel_sharpening)
        psnr_amsharp = PSNR(image,amsharp_image)
        ssim_amsharp = ssim(image,amsharp_image,multichannel=True)
        mse_amsharp = mse(image,amsharp_image)

        # gray scale images
        bwimage = cv2.imread(os.path.join(dir,f),cv2.IMREAD_GRAYSCALE)
        # Laplacian Filter 
        dest = cv2.Laplacian(bwimage, cv2.CV_16S, ksize=3)
        laplacian_image = cv2.convertScaleAbs(dest)
        psnr_laplacian = PSNR(bwimage,laplacian_image)
        ssim_laplacian = ssim(bwimage,laplacian_image)
        mse_laplacian = mse(bwimage,laplacian_image)

        # LoG Filter
        gaussian_image = cv2.GaussianBlur(bwimage,(5,5),0)
        dest = cv2.Laplacian(gaussian_image, cv2.CV_64F, ksize=3)
        log_image = cv2.convertScaleAbs(dest)
        psnr_log = PSNR(bwimage,log_image)
        ssim_log = ssim(bwimage,log_image)
        mse_log = mse(bwimage,log_image)
        
        # Adaptive Median Filter
        admed_image = AdaptiveMedianFilter(bwimage)
        psnr_admed = PSNR(bwimage,admed_image)
        ssim_admed = ssim(bwimage,admed_image)
        mse_admed = mse(bwimage,admed_image)
        
        # Midpoint Filter
        midpoint_image = midpoint(bwimage)
        psnr_midpoint = PSNR(bwimage,midpoint_image)
        ssim_midpoint = ssim(bwimage,midpoint_image)
        mse_midpoint = mse(bwimage,midpoint_image)
        
        # Gabor Filter
        gabor_image = Gabor_process(bwimage)
        psnr_gabor = PSNR(bwimage,gabor_image)
        ssim_gabor = ssim(bwimage,gabor_image)
        mse_gabor = mse(bwimage,gabor_image)
        
        writer3.writerow([psnr_mean,psnr_median,psnr_gaussian,psnr_unsharp,psnr_wiener,psnr_chmean,psnr_amsharp])
        writer33.writerow([psnr_laplacian,psnr_log,psnr_admed,psnr_midpoint,psnr_gabor])
        swriter3.writerow([ssim_mean,ssim_median,ssim_gaussian,ssim_unsharp,ssim_wiener,ssim_chmean,ssim_amsharp])
        swriter33.writerow([ssim_laplacian,ssim_log,ssim_admed,ssim_midpoint,ssim_gabor])
        mwriter3.writerow([mse_mean,mse_median,mse_gaussian,mse_unsharp,mse_wiener,mse_chmean,mse_amsharp])
        mwriter33.writerow([mse_laplacian,mse_log,mse_admed,mse_midpoint,mse_gabor])
      
# Fusarium Data set
      elif(f.startswith("fusarium")):
        image = cv2.imread(os.path.join(dir,f))
        # mean filter
        kernel = np.ones((3,3),np.float32)/9
        mean_image = cv2.filter2D(image,-1,kernel)
        psnr_mean = PSNR(image,mean_image)
        ssim_mean = ssim(image,mean_image,multichannel=True)
        mse_mean = mse(image,mean_image)

        # median filter
        median_image = cv2.medianBlur(image,9)
        psnr_median = PSNR(image,median_image)
        ssim_median = ssim(image,median_image,multichannel=True)
        mse_median = mse(image,median_image)
        
        # gaussian filter
        gaussian_image = cv2.GaussianBlur(image,(5,5),0)
        psnr_gaussian = PSNR(image,gaussian_image)
        ssim_gaussian = ssim(image,gaussian_image,multichannel=True)
        mse_gaussian = mse(image,gaussian_image)
        
        # unsharp mask filter
        sharp = np.zeros_like(image)
        for i in range(3):
          sharp[:,:,i] = unsharp(image[:,:,i], 5, 0.8)
        psnr_unsharp = PSNR(image,sharp)
        ssim_unsharp = ssim(image,sharp,multichannel=True)
        mse_unsharp = mse(image,sharp)
        
        # wiener filter
        wiener_image = np.ones_like(image)
        for i in range(3):
          wiener_image[:,:,i] = weiner_filter(wiener_image[:,:,i])
        psnr_wiener = PSNR(image,wiener_image)
        ssim_wiener = ssim(image,wiener_image,multichannel=True)
        mse_wiener = mse(image,wiener_image)

        # contraharmonic mean filter
        chmean_image = contraharmonic_mean(image,(3,3),0.5)
        psnr_chmean = PSNR(image,chmean_image)
        ssim_chmean = ssim(image,chmean_image,multichannel=True)
        mse_chmean = mse(image,chmean_image)

        # Arithmetic filter with sharpening kernel
        kernel_sharpening = np.array([[-1,-1,-1],
                              [-1, 9,-1],
                              [-1,-1,-1]])
        amsharp_image = cv2.filter2D(image, -1, kernel_sharpening)
        psnr_amsharp = PSNR(image,amsharp_image)
        ssim_amsharp = ssim(image,amsharp_image,multichannel=True)
        mse_amsharp = mse(image,amsharp_image)

        # gray scale images
        bwimage = cv2.imread(os.path.join(dir,f),cv2.IMREAD_GRAYSCALE)
        # Laplacian Filter 
        dest = cv2.Laplacian(bwimage, cv2.CV_16S, ksize=3)
        laplacian_image = cv2.convertScaleAbs(dest)
        psnr_laplacian = PSNR(bwimage,laplacian_image)
        ssim_laplacian = ssim(bwimage,laplacian_image)
        mse_laplacian = mse(bwimage,laplacian_image)

        # LoG Filter
        gaussian_image = cv2.GaussianBlur(bwimage,(5,5),0)
        dest = cv2.Laplacian(gaussian_image, cv2.CV_64F, ksize=3)
        log_image = cv2.convertScaleAbs(dest)
        psnr_log = PSNR(bwimage,log_image)
        ssim_log = ssim(bwimage,log_image)
        mse_log = mse(bwimage,log_image)
        
        # Adaptive Median Filter
        admed_image = AdaptiveMedianFilter(bwimage)
        psnr_admed = PSNR(bwimage,admed_image)
        ssim_admed = ssim(bwimage,admed_image)
        mse_admed = mse(bwimage,admed_image)
        
        # Midpoint Filter
        midpoint_image = midpoint(bwimage)
        psnr_midpoint = PSNR(bwimage,midpoint_image)
        ssim_midpoint = ssim(bwimage,midpoint_image)
        mse_midpoint = mse(bwimage,midpoint_image)
        
        # Gabor Filter
        gabor_image = Gabor_process(bwimage)
        psnr_gabor = PSNR(bwimage,gabor_image)
        ssim_gabor = ssim(bwimage,gabor_image)
        mse_gabor = mse(bwimage,gabor_image)
        
        writer4.writerow([psnr_mean,psnr_median,psnr_gaussian,psnr_unsharp,psnr_wiener,psnr_chmean,psnr_amsharp])
        writer44.writerow([psnr_laplacian,psnr_log,psnr_admed,psnr_midpoint,psnr_gabor])
        swriter4.writerow([ssim_mean,ssim_median,ssim_gaussian,ssim_unsharp,ssim_wiener,ssim_chmean,ssim_amsharp])
        swriter44.writerow([ssim_laplacian,ssim_log,ssim_admed,ssim_midpoint,ssim_gabor])
        mwriter4.writerow([mse_mean,mse_median,mse_gaussian,mse_unsharp,mse_wiener,mse_chmean,mse_amsharp])
        mwriter44.writerow([mse_laplacian,mse_log,mse_admed,mse_midpoint,mse_gabor])
      
# Bacterial Leaf Spot Data Set
      elif(f.startswith("bacterial")):
        image = cv2.imread(os.path.join(dir,f))
        # mean filter
        kernel = np.ones((3,3),np.float32)/9
        mean_image = cv2.filter2D(image,-1,kernel)
        psnr_mean = PSNR(image,mean_image)
        ssim_mean = ssim(image,mean_image,multichannel=True)
        mse_mean = mse(image,mean_image)

        # median filter
        median_image = cv2.medianBlur(image,9)
        psnr_median = PSNR(image,median_image)
        ssim_median = ssim(image,median_image,multichannel=True)
        mse_median = mse(image,median_image)
        
        # gaussian filter
        gaussian_image = cv2.GaussianBlur(image,(5,5),0)
        psnr_gaussian = PSNR(image,gaussian_image)
        ssim_gaussian = ssim(image,gaussian_image,multichannel=True)
        mse_gaussian = mse(image,gaussian_image)
        
        # unsharp mask filter
        sharp = np.zeros_like(image)
        for i in range(3):
          sharp[:,:,i] = unsharp(image[:,:,i], 5, 0.8)
        psnr_unsharp = PSNR(image,sharp)
        ssim_unsharp = ssim(image,sharp,multichannel=True)
        mse_unsharp = mse(image,sharp)
        
        # wiener filter
        wiener_image = np.ones_like(image)
        for i in range(3):
          wiener_image[:,:,i] = weiner_filter(wiener_image[:,:,i])
        psnr_wiener = PSNR(image,wiener_image)
        ssim_wiener = ssim(image,wiener_image,multichannel=True)
        mse_wiener = mse(image,wiener_image)

        # contraharmonic mean filter
        chmean_image = contraharmonic_mean(image,(3,3),0.5)
        psnr_chmean = PSNR(image,chmean_image)
        ssim_chmean = ssim(image,chmean_image,multichannel=True)
        mse_chmean = mse(image,chmean_image)

        # Arithmetic filter with sharpening kernel
        kernel_sharpening = np.array([[-1,-1,-1],
                              [-1, 9,-1],
                              [-1,-1,-1]])
        amsharp_image = cv2.filter2D(image, -1, kernel_sharpening)
        psnr_amsharp = PSNR(image,amsharp_image)
        ssim_amsharp = ssim(image,amsharp_image,multichannel=True)
        mse_amsharp = mse(image,amsharp_image)

        # gray scale images
        bwimage = cv2.imread(os.path.join(dir,f),cv2.IMREAD_GRAYSCALE)
        # Laplacian Filter 
        dest = cv2.Laplacian(bwimage, cv2.CV_16S, ksize=3)
        laplacian_image = cv2.convertScaleAbs(dest)
        psnr_laplacian = PSNR(bwimage,laplacian_image)
        ssim_laplacian = ssim(bwimage,laplacian_image)
        mse_laplacian = mse(bwimage,laplacian_image)

        # LoG Filter
        gaussian_image = cv2.GaussianBlur(bwimage,(5,5),0)
        dest = cv2.Laplacian(gaussian_image, cv2.CV_64F, ksize=3)
        log_image = cv2.convertScaleAbs(dest)
        psnr_log = PSNR(bwimage,log_image)
        ssim_log = ssim(bwimage,log_image)
        mse_log = mse(bwimage,log_image)
        
        # Adaptive Median Filter
        admed_image = AdaptiveMedianFilter(bwimage)
        psnr_admed = PSNR(bwimage,admed_image)
        ssim_admed = ssim(bwimage,admed_image)
        mse_admed = mse(bwimage,admed_image)
        
        # Midpoint Filter
        midpoint_image = midpoint(bwimage)
        psnr_midpoint = PSNR(bwimage,midpoint_image)
        ssim_midpoint = ssim(bwimage,midpoint_image)
        mse_midpoint = mse(bwimage,midpoint_image)
        
        # Gabor Filter
        gabor_image = Gabor_process(bwimage)
        psnr_gabor = PSNR(bwimage,gabor_image)
        ssim_gabor = ssim(bwimage,gabor_image)
        mse_gabor = mse(bwimage,gabor_image)
        
        writer5.writerow([psnr_mean,psnr_median,psnr_gaussian,psnr_unsharp,psnr_wiener,psnr_chmean,psnr_amsharp])
        writer55.writerow([psnr_laplacian,psnr_log,psnr_admed,psnr_midpoint,psnr_gabor])
        swriter5.writerow([ssim_mean,ssim_median,ssim_gaussian,ssim_unsharp,ssim_wiener,ssim_chmean,ssim_amsharp])
        swriter55.writerow([ssim_laplacian,ssim_log,ssim_admed,ssim_midpoint,ssim_gabor])
        mwriter5.writerow([mse_mean,mse_median,mse_gaussian,mse_unsharp,mse_wiener,mse_chmean,mse_amsharp])
        mwriter55.writerow([mse_laplacian,mse_log,mse_admed,mse_midpoint,mse_gabor])



# Creating Final Filtering data

In [ ]:
for dir in folders:
  for _,_,files in os.walk(dir):
    for f in files:

# curl data set      
      if(f.startswith("curl_")):
        image = cv2.imread(os.path.join(dir,f))
        # unsharp mask filter
        sharp = np.zeros_like(image)
        for i in range(3):
          sharp[:,:,i] = unsharp(image[:,:,i], 5, 0.8)

        # gray scale images
        bwimage = cv2.imread(os.path.join(dir,f),cv2.IMREAD_GRAYSCALE)
        # Laplacian Filter 
        dest = cv2.Laplacian(bwimage, cv2.CV_16S, ksize=3)
        laplacian_image = cv2.convertScaleAbs(dest)
        cv2.imwrite('/content/drive/MyDrive/Image_Processing_Plants/Filtered_Data/Curl-Unsharp/unsharp_'+f,sharp)
        cv2.imwrite('/content/drive/MyDrive/Image_Processing_Plants/Filtered_Data/Curl-Laplacian/laplacian_'+f,laplacian_image)

# Pests Data Set      
      elif(f.startswith("pests_")): 
        image = cv2.imread(os.path.join(dir,f))
        
        # unsharp mask filter
        sharp = np.zeros_like(image)
        for i in range(3):
          sharp[:,:,i] = unsharp(image[:,:,i], 5, 0.8)

        # gray scale images
        bwimage = cv2.imread(os.path.join(dir,f),cv2.IMREAD_GRAYSCALE)
        # Laplacian Filter 
        dest = cv2.Laplacian(bwimage, cv2.CV_16S, ksize=3)
        laplacian_image = cv2.convertScaleAbs(dest)
        cv2.imwrite('/content/drive/MyDrive/Image_Processing_Plants/Filtered_Data/Pests-Unsharp/unsharp_'+f,sharp)
        cv2.imwrite('/content/drive/MyDrive/Image_Processing_Plants/Filtered_Data/Pests-Laplacian/laplacian_'+f,laplacian_image)

# Healthy Data set      
      elif(f.startswith("healthy")):
        image = cv2.imread(os.path.join(dir,f))

        # unsharp mask filter
        sharp = np.zeros_like(image)
        for i in range(3):
          sharp[:,:,i] = unsharp(image[:,:,i], 5, 0.8)

        # gray scale images
        bwimage = cv2.imread(os.path.join(dir,f),cv2.IMREAD_GRAYSCALE)
        # Laplacian Filter 
        dest = cv2.Laplacian(bwimage, cv2.CV_16S, ksize=3)
        laplacian_image = cv2.convertScaleAbs(dest)
        cv2.imwrite('/content/drive/MyDrive/Image_Processing_Plants/Filtered_Data/Healthy-Unsharp/unsharp_'+f,sharp)
        cv2.imwrite('/content/drive/MyDrive/Image_Processing_Plants/Filtered_Data/Healthy-Laplacian/laplacian_'+f,laplacian_image)

      
# Fusarium Data set
      elif(f.startswith("fusarium")):
        image = cv2.imread(os.path.join(dir,f))
        
        # unsharp mask filter
        sharp = np.zeros_like(image)
        for i in range(3):
          sharp[:,:,i] = unsharp(image[:,:,i], 5, 0.8)
        
        # gray scale images
        bwimage = cv2.imread(os.path.join(dir,f),cv2.IMREAD_GRAYSCALE)
        # Laplacian Filter 
        dest = cv2.Laplacian(bwimage, cv2.CV_16S, ksize=3)
        laplacian_image = cv2.convertScaleAbs(dest)
        cv2.imwrite('/content/drive/MyDrive/Image_Processing_Plants/Filtered_Data/Fusarium-Unsharp/unsharp_'+f,sharp)
        cv2.imwrite('/content/drive/MyDrive/Image_Processing_Plants/Filtered_Data/Fusarium-Laplacian/laplacian_'+f,laplacian_image)

      
# Bacterial Leaf Spot Data Set
      elif(f.startswith("bacterial")):
        image = cv2.imread(os.path.join(dir,f))
        
        # unsharp mask filter
        sharp = np.zeros_like(image)
        for i in range(3):
          sharp[:,:,i] = unsharp(image[:,:,i], 5, 0.8)
        
        # gray scale images
        bwimage = cv2.imread(os.path.join(dir,f),cv2.IMREAD_GRAYSCALE)
        # Laplacian Filter 
        dest = cv2.Laplacian(bwimage, cv2.CV_16S, ksize=3)
        laplacian_image = cv2.convertScaleAbs(dest)
        cv2.imwrite('/content/drive/MyDrive/Image_Processing_Plants/Filtered_Data/Bacterial-Leaf-Spot-Unsharp/unsharp_'+f,sharp)
        cv2.imwrite('/content/drive/MyDrive/Image_Processing_Plants/Filtered_Data/Bacterial-Leaf-Spot/laplacian_'+f,laplacian_image)




# Public Dataset

In [ ]:
path = '/content/drive/MyDrive/Image_Processing_Plants/Public Dataset'

folders = []
for root,dirs,files in os.walk(path):
  for dir in dirs:
    folders.append(os.path.join(root,dir))

In [ ]:
psnr_head = ['psnr_mean','psnr_median','psnr_gaussian','psnr_unsharp','psnr_wiener','psnr_chmean','psnr_amsharp',
           'psnr_laplacian','psnr_log','psnr_admed','psnr_midpoint','psnr_gabor']
ssim_head = ['ssim_mean','ssim_median','ssim_gaussian','ssim_unsharp','ssim_wiener','ssim_chmean','ssim_amsharp',
             'ssim_laplacian','ssim_log','ssim_admed','ssim_midpoint','ssim_gabor']
mse_head = ['mse_mean','mse_median','mse_gaussian','mse_unsharp','mse_wiener','mse_chmean','mse_amsharp',
             'mse_laplacian','mse_log','mse_admed','mse_midpoint','mse_gabor']

df_HL_psnr = pd.DataFrame(columns=psnr_head)
df_HL_ssim = pd.DataFrame(columns=ssim_head)
df_HL_mse = pd.DataFrame(columns=mse_head)

df_D_psnr = pd.DataFrame(columns=psnr_head)
df_D_ssim = pd.DataFrame(columns=ssim_head)
df_D_mse = pd.DataFrame(columns=mse_head)

for dir in folders:
  for _,_,files in os.walk(dir):
    for f in files:

# Diseases data set      
      if("B.Spot" in f):
        image = cv2.imread(os.path.join(dir,f))
        # mean filter
        kernel = np.ones((3,3),np.float32)/9
        mean_image = cv2.filter2D(image,-1,kernel)
        psnr_mean = PSNR(image,mean_image)
        ssim_mean = ssim(image,mean_image,multichannel=True)
        mse_mean = mse(image,mean_image)

        # median filter
        median_image = cv2.medianBlur(image,9)
        psnr_median = PSNR(image,median_image)
        ssim_median = ssim(image,median_image,multichannel=True)
        mse_median = mse(image,median_image)
        
        # gaussian filter
        gaussian_image = cv2.GaussianBlur(image,(5,5),0)
        psnr_gaussian = PSNR(image,gaussian_image)
        ssim_gaussian = ssim(image,gaussian_image,multichannel=True)
        mse_gaussian = mse(image,gaussian_image)
        
        # unsharp mask filter
        sharp = np.zeros_like(image)
        for i in range(3):
          sharp[:,:,i] = unsharp(image[:,:,i], 5, 0.8)
        psnr_unsharp = PSNR(image,sharp)
        ssim_unsharp = ssim(image,sharp,multichannel=True)
        mse_unsharp = mse(image,sharp)
        
        # wiener filter
        wiener_image = np.ones_like(image)
        for i in range(3):
          wiener_image[:,:,i] = weiner_filter(wiener_image[:,:,i])
        psnr_wiener = PSNR(image,wiener_image)
        ssim_wiener = ssim(image,wiener_image,multichannel=True)
        mse_wiener = mse(image,wiener_image)

        # contraharmonic mean filter
        chmean_image = contraharmonic_mean(image,(3,3),0.5)
        psnr_chmean = PSNR(image,chmean_image)
        ssim_chmean = ssim(image,chmean_image,multichannel=True)
        mse_chmean = mse(image,chmean_image)

        # Arithmetic filter with sharpening kernel
        kernel_sharpening = np.array([[-1,-1,-1],
                              [-1, 9,-1],
                              [-1,-1,-1]])
        amsharp_image = cv2.filter2D(image, -1, kernel_sharpening)
        psnr_amsharp = PSNR(image,amsharp_image)
        ssim_amsharp = ssim(image,amsharp_image,multichannel=True)
        mse_amsharp = mse(image,amsharp_image)

        # gray scale images
        bwimage = cv2.imread(os.path.join(dir,f),cv2.IMREAD_GRAYSCALE)
        # Laplacian Filter 
        dest = cv2.Laplacian(bwimage, cv2.CV_16S, ksize=3)
        laplacian_image = cv2.convertScaleAbs(dest)
        psnr_laplacian = PSNR(bwimage,laplacian_image)
        ssim_laplacian = ssim(bwimage,laplacian_image)
        mse_laplacian = mse(bwimage,laplacian_image)

        # LoG Filter
        gaussian_image = cv2.GaussianBlur(bwimage,(5,5),0)
        dest = cv2.Laplacian(gaussian_image, cv2.CV_64F, ksize=3)
        log_image = cv2.convertScaleAbs(dest)
        psnr_log = PSNR(bwimage,log_image)
        ssim_log = ssim(bwimage,log_image)
        mse_log = mse(bwimage,log_image)
        
        # Adaptive Median Filter
        admed_image = AdaptiveMedianFilter(bwimage)
        psnr_admed = PSNR(bwimage,admed_image)
        ssim_admed = ssim(bwimage,admed_image)
        mse_admed = mse(bwimage,admed_image)
        
        # Midpoint Filter
        midpoint_image = midpoint(bwimage)
        psnr_midpoint = PSNR(bwimage,midpoint_image)
        ssim_midpoint = ssim(bwimage,midpoint_image)
        mse_midpoint = mse(bwimage,midpoint_image)
        
        # Gabor Filter
        gabor_image = Gabor_process(bwimage)
        psnr_gabor = PSNR(bwimage,gabor_image)
        ssim_gabor = ssim(bwimage,gabor_image)
        mse_gabor = mse(bwimage,gabor_image)
        
        df_HL_psnr = df_HL_psnr.append({'psnr_mean':psnr_mean,'psnr_median':psnr_median,
                                        'psnr_gaussian':psnr_gaussian,'psnr_unsharp':psnr_unsharp,
                                        'psnr_wiener':psnr_wiener,'psnr_chmean':psnr_chmean,'psnr_amsharp':psnr_amsharp,
                                        'psnr_laplacian':psnr_laplacian,'psnr_log':psnr_log,'psnr_admed':psnr_admed,
                                        'psnr_midpoint':psnr_midpoint,'psnr_gabor':psnr_gabor},ignore_index=True)
        df_HL_ssim = df_HL_ssim.append({'ssim_mean':ssim_mean,'ssim_median':ssim_median,
                                        'ssim_gaussian':ssim_gaussian,'ssim_unsharp':ssim_unsharp,
                                        'ssim_wiener':ssim_wiener,'ssim_chmean':ssim_chmean,'ssim_amsharp':ssim_amsharp,
                                        'ssim_laplacian':ssim_laplacian,'ssim_log':ssim_log,'ssim_admed':ssim_admed,
                                        'ssim_midpoint':ssim_midpoint,'ssim_gabor':ssim_gabor},ignore_index=True)
        df_HL_mse = df_HL_mse.append({'mse_mean':mse_mean,'mse_median':mse_median,
                                        'mse_gaussian':mse_gaussian,'mse_unsharp':mse_unsharp,
                                        'mse_wiener':mse_wiener,'mse_chmean':mse_chmean,'mse_amsharp':mse_amsharp,
                                        'mse_laplacian':mse_laplacian,'mse_log':mse_log,'mse_admed':mse_admed,
                                        'mse_midpoint':mse_midpoint,'mse_gabor':mse_gabor},ignore_index=True)
        print(".",end="")

# Healthy Data Set      
      elif("HL" in f): 
        image = cv2.imread(os.path.join(dir,f))
        # mean filter
        kernel = np.ones((3,3),np.float32)/9
        mean_image = cv2.filter2D(image,-1,kernel)
        psnr_mean = PSNR(image,mean_image)
        ssim_mean = ssim(image,mean_image,multichannel=True)
        mse_mean = mse(image,mean_image)

        # median filter
        median_image = cv2.medianBlur(image,9)
        psnr_median = PSNR(image,median_image)
        ssim_median = ssim(image,median_image,multichannel=True)
        mse_median = mse(image,median_image)
        
        # gaussian filter
        gaussian_image = cv2.GaussianBlur(image,(5,5),0)
        psnr_gaussian = PSNR(image,gaussian_image)
        ssim_gaussian = ssim(image,gaussian_image,multichannel=True)
        mse_gaussian = mse(image,gaussian_image)
        
        # unsharp mask filter
        sharp = np.zeros_like(image)
        for i in range(3):
          sharp[:,:,i] = unsharp(image[:,:,i], 5, 0.8)
        psnr_unsharp = PSNR(image,sharp)
        ssim_unsharp = ssim(image,sharp,multichannel=True)
        mse_unsharp = mse(image,sharp)
        
        # wiener filter
        wiener_image = np.ones_like(image)
        for i in range(3):
          wiener_image[:,:,i] = weiner_filter(wiener_image[:,:,i])
        psnr_wiener = PSNR(image,wiener_image)
        ssim_wiener = ssim(image,wiener_image,multichannel=True)
        mse_wiener = mse(image,wiener_image)

        # contraharmonic mean filter
        chmean_image = contraharmonic_mean(image,(3,3),0.5)
        psnr_chmean = PSNR(image,chmean_image)
        ssim_chmean = ssim(image,chmean_image,multichannel=True)
        mse_chmean = mse(image,chmean_image)

        # Arithmetic filter with sharpening kernel
        kernel_sharpening = np.array([[-1,-1,-1],
                              [-1, 9,-1],
                              [-1,-1,-1]])
        amsharp_image = cv2.filter2D(image, -1, kernel_sharpening)
        psnr_amsharp = PSNR(image,amsharp_image)
        ssim_amsharp = ssim(image,amsharp_image,multichannel=True)
        mse_amsharp = mse(image,amsharp_image)

        # gray scale images
        bwimage = cv2.imread(os.path.join(dir,f),cv2.IMREAD_GRAYSCALE)
        # Laplacian Filter 
        dest = cv2.Laplacian(bwimage, cv2.CV_16S, ksize=3)
        laplacian_image = cv2.convertScaleAbs(dest)
        psnr_laplacian = PSNR(bwimage,laplacian_image)
        ssim_laplacian = ssim(bwimage,laplacian_image)
        mse_laplacian = mse(bwimage,laplacian_image)

        # LoG Filter
        gaussian_image = cv2.GaussianBlur(bwimage,(5,5),0)
        dest = cv2.Laplacian(gaussian_image, cv2.CV_64F, ksize=3)
        log_image = cv2.convertScaleAbs(dest)
        psnr_log = PSNR(bwimage,log_image)
        ssim_log = ssim(bwimage,log_image)
        mse_log = mse(bwimage,log_image)
        
        # Adaptive Median Filter
        admed_image = AdaptiveMedianFilter(bwimage)
        psnr_admed = PSNR(bwimage,admed_image)
        ssim_admed = ssim(bwimage,admed_image)
        mse_admed = mse(bwimage,admed_image)
        
        # Midpoint Filter
        midpoint_image = midpoint(bwimage)
        psnr_midpoint = PSNR(bwimage,midpoint_image)
        ssim_midpoint = ssim(bwimage,midpoint_image)
        mse_midpoint = mse(bwimage,midpoint_image)
        
        # Gabor Filter
        gabor_image = Gabor_process(bwimage)
        psnr_gabor = PSNR(bwimage,gabor_image)
        ssim_gabor = ssim(bwimage,gabor_image)
        mse_gabor = mse(bwimage,gabor_image)
        df_D_psnr = df_D_psnr.append({'psnr_mean':psnr_mean,'psnr_median':psnr_median,
                                        'psnr_gaussian':psnr_gaussian,'psnr_unsharp':psnr_unsharp,
                                        'psnr_wiener':psnr_wiener,'psnr_chmean':psnr_chmean,'psnr_amsharp':psnr_amsharp,
                                        'psnr_laplacian':psnr_laplacian,'psnr_log':psnr_log,'psnr_admed':psnr_admed,
                                        'psnr_midpoint':psnr_midpoint,'psnr_gabor':psnr_gabor},ignore_index=True)
        df_D_ssim = df_D_ssim.append({'ssim_mean':ssim_mean,'ssim_median':ssim_median,
                                        'ssim_gaussian':ssim_gaussian,'ssim_unsharp':ssim_unsharp,
                                        'ssim_wiener':ssim_wiener,'ssim_chmean':ssim_chmean,'ssim_amsharp':ssim_amsharp,
                                        'ssim_laplacian':ssim_laplacian,'ssim_log':ssim_log,'ssim_admed':ssim_admed,
                                        'ssim_midpoint':ssim_midpoint,'ssim_gabor':ssim_gabor},ignore_index=True)
        df_D_mse = df_D_mse.append({'mse_mean':mse_mean,'mse_median':mse_median,
                                        'mse_gaussian':mse_gaussian,'mse_unsharp':mse_unsharp,
                                        'mse_wiener':mse_wiener,'mse_chmean':mse_chmean,'mse_amsharp':mse_amsharp,
                                        'mse_laplacian':mse_laplacian,'mse_log':mse_log,'mse_admed':mse_admed,
                                        'mse_midpoint':mse_midpoint,'mse_gabor':mse_gabor},ignore_index=True)
        


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in true_divide
  """
/usr/local/lib/python3.7/dist-packages/skimage/metrics/_structural_similarity.py:108: UserWarning: Inputs have mismatched dtype.  Setting data_range based on im1.dtype.
  im2[..., ch], **args)
/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:93: UserWarning: Inputs have mismatched dtype.  Setting data_range based on im1.dtype.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:99: UserWarning: Inputs have mismatched dtype.  Setting data_range based on im1.dtype.


.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in true_divide
  """
/usr/local/lib/python3.7/dist-packages/skimage/metrics/_structural_similarity.py:108: UserWarning: Inputs have mismatched dtype.  Setting data_range based on im1.dtype.
  im2[..., ch], **args)


....................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:198: UserWarning: Inputs have mismatched dtype.  Setting data_range based on im1.dtype.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:204: UserWarning: Inputs have mismatched dtype.  Setting data_range based on im1.dtype.


In [ ]:
df_HL_psnr.describe()

,psnr_mean,psnr_median,psnr_gaussian,psnr_unsharp,psnr_wiener,psnr_chmean,psnr_amsharp,psnr_laplacian,psnr_log,psnr_admed,psnr_midpoint,psnr_gabor
count,997.000000,997.000000,997.000000,997.000000,997.000000,237.000000,997.000000,997.000000,997.000000,997.000000,997.000000,997.000000
mean,30.566188,30.026122,30.610390,33.338675,27.851538,24.613762,28.183998,27.945350,27.915157,27.209070,8.825613,28.004190
std,1.260768,0.795270,1.179777,0.924865,0.157975,1.816618,0.240567,0.071158,0.169402,2.622331,0.773129,0.257103
min,28.484172,28.408111,28.513492,30.817339,27.479206,20.285293,27.834183,27.683065,27.481274,21.208812,6.168357,27.457492
25%,29.774437,29.499400,29.856478,32.734168,27.740285,23.383306,28.010809,27.908095,27.801935,25.599149,8.398406,27.823284
50%,30.237722,29.858736,30.330210,33.244591,27.853289,24.394335,28.140179,27.939792,27.910678,26.704618,8.845477,27.959307
75%,30.937683,30.363919,30.997492,33.863187,27.955694,25.660078,28.287811,27.973833,28.002138,28.309440,9.235455,28.124406
max,39.267353,35.020893,38.590288,38.336099,28.454091,33.553951,29.636348,28.601098,28.862989,42.072200,12.750644,29.433347


In [ ]:
df_HL_ssim.describe()

,ssim_mean,ssim_median,ssim_gaussian,ssim_unsharp,ssim_wiener,ssim_chmean,ssim_amsharp,ssim_laplacian,ssim_log,ssim_admed,ssim_midpoint,ssim_gabor
count,997.000000,997.000000,997.000000,997.000000,997.000000,237.000000,997.000000,997.000000,997.000000,997.000000,997.000000,997.000000
mean,0.630274,0.459133,0.654082,0.935841,0.002883,0.601290,0.368645,0.056340,0.063905,0.816358,0.247451,0.273877
std,0.099826,0.099018,0.088589,0.003355,0.002636,0.091462,0.035872,0.019539,0.024136,0.054227,0.078142,0.041190
min,0.373056,0.197242,0.419642,0.928174,0.000077,0.418364,0.296266,0.004545,0.009983,0.682674,0.017093,0.143910
25%,0.560071,0.390905,0.592838,0.933659,0.000856,0.540891,0.341282,0.042964,0.045968,0.778666,0.197196,0.244687
50%,0.611303,0.440153,0.637279,0.935438,0.002161,0.580795,0.368183,0.054847,0.062698,0.806961,0.243297,0.274536
75%,0.686385,0.509413,0.703182,0.937759,0.004166,0.635690,0.391109,0.068667,0.081462,0.847702,0.290430,0.302032
max,0.959447,0.832758,0.951763,0.956664,0.015756,0.908330,0.539674,0.123266,0.137797,0.986425,0.596733,0.434418


In [ ]:
df_HL_mse.describe()

,mse_mean,mse_median,mse_gaussian,mse_unsharp,mse_wiener,mse_chmean,mse_amsharp,mse_laplacian,mse_log,mse_admed,mse_midpoint,mse_gabor
count,997.000000,997.000000,997.000000,997.000000,997.000000,237.000000,997.000000,997.000000,997.000000,997.000000,997.000000,997.000000
mean,250.518828,362.481859,232.463664,106.365949,16112.653061,242.626795,4724.519770,9334.687771,13329.186899,142.189513,8657.212509,11535.707871
std,107.271325,135.928359,96.886740,25.397196,2682.173013,89.269027,1150.356513,1881.788554,2717.949491,66.766383,1561.911435,2195.556711
min,12.270335,40.359924,14.904897,22.158081,9954.178055,28.687008,557.730296,5930.170792,7918.088852,4.035156,3451.564445,6577.004105
25%,177.798665,267.826681,167.784780,90.157069,14436.842097,176.633287,4128.515071,8143.783646,11589.183212,95.970200,7754.161449,10199.959229
50%,247.628779,354.280101,229.756231,105.758692,15710.891871,236.399667,4948.427266,9037.571625,12920.949158,138.873245,8482.672585,11249.385208
75%,314.044800,447.374964,290.654887,122.460434,17281.597229,298.366303,5530.535756,10077.359360,14498.363388,179.128799,9402.421562,12559.692886
max,781.735092,971.053991,704.781499,186.029882,37852.785263,608.907084,7490.721252,29218.521149,36071.563995,492.265228,15712.478035,29371.547134


In [ ]:
df_D_psnr.describe()

,psnr_mean,psnr_median,psnr_gaussian,psnr_unsharp,psnr_wiener,psnr_chmean,psnr_amsharp,psnr_laplacian,psnr_log,psnr_admed,psnr_midpoint,psnr_gabor
count,1476.000000,1476.000000,1476.000000,1476.000000,1476.000000,777.000000,1476.000000,1476.000000,1476.000000,1476.000000,1476.000000,1476.000000
mean,31.160882,30.447288,31.198796,34.062755,27.827195,26.224150,28.138953,27.869276,27.872973,28.613506,8.870597,27.838249
std,1.786536,1.193660,1.712408,1.464472,0.120911,3.451616,0.265318,0.076413,0.152535,3.699007,1.158944,0.223765
min,28.539976,28.533817,28.592604,30.848845,27.460949,19.449503,27.787426,27.551510,27.417803,20.902338,6.200366,27.297262
25%,29.861550,29.565006,29.957710,33.007788,27.742081,23.661921,27.954856,27.825437,27.766946,25.920255,8.120095,27.691857
50%,30.778954,30.215152,30.851298,33.936791,27.816532,25.447470,28.060513,27.875691,27.876410,27.990108,8.703098,27.840000
75%,32.133904,31.164579,32.182063,34.973976,27.900066,28.772527,28.244824,27.916661,27.972594,30.995622,9.479042,27.981766
max,41.739440,35.903601,40.525218,38.688961,28.552474,40.812843,30.009646,28.341633,28.454715,45.231790,20.064748,28.694587


In [ ]:
df_D_ssim.describe()

,ssim_mean,ssim_median,ssim_gaussian,ssim_unsharp,ssim_wiener,ssim_chmean,ssim_amsharp,ssim_laplacian,ssim_log,ssim_admed,ssim_midpoint,ssim_gabor
count,1476.000000,1476.000000,1476.000000,1476.000000,1476.000000,777.000000,1476.000000,1476.000000,1476.000000,1476.000000,1476.000000,1476.000000
mean,0.641709,0.483282,0.664181,0.937200,0.001228,0.641262,0.355571,0.043317,0.046361,0.821337,0.274171,0.276038
std,0.133601,0.135850,0.119596,0.004446,0.001545,0.138006,0.033726,0.013341,0.014191,0.072339,0.118417,0.039386
min,0.345469,0.188928,0.396705,0.926660,0.000048,0.340990,0.280439,-0.001723,0.000259,0.660184,0.039387,0.141241
25%,0.542234,0.380695,0.575399,0.934236,0.000365,0.538697,0.333019,0.033845,0.036664,0.766862,0.186763,0.249499
50%,0.635134,0.472743,0.659382,0.936319,0.000652,0.634685,0.349857,0.041835,0.044198,0.817658,0.257318,0.270743
75%,0.741787,0.579493,0.752841,0.939548,0.001499,0.753312,0.374725,0.051512,0.054261,0.876090,0.352081,0.299345
max,0.975627,0.872679,0.969723,0.957469,0.024052,0.975390,0.550773,0.096919,0.107300,0.992584,0.703887,0.422739


In [ ]:
df_D_mse.describe()

,mse_mean,mse_median,mse_gaussian,mse_unsharp,mse_wiener,mse_chmean,mse_amsharp,mse_laplacian,mse_log,mse_admed,mse_midpoint,mse_gabor
count,1476.000000,1476.000000,1476.000000,1476.000000,1476.000000,777.000000,1476.000000,1476.000000,1476.000000,1476.000000,1476.000000,1476.000000
mean,207.514029,282.457720,192.495742,88.400426,18304.906990,201.889834,4308.834723,10444.094933,15676.576564,120.670980,8710.529678,12882.161909
std,137.767464,168.723133,124.478755,34.456392,4116.098252,134.268683,1576.850770,2602.093101,3870.973599,86.330209,2092.770898,3315.451768
min,4.455444,21.123322,6.030034,15.921010,7990.892456,5.392573,298.525274,4775.408234,6661.866608,1.949402,640.627415,5130.349609
25%,98.144736,155.150911,93.804415,60.446548,15383.739634,86.263624,3129.478600,8603.846012,13050.851337,51.703289,7331.218938,10500.588318
50%,183.540693,249.516584,171.408806,90.340126,17588.233869,185.495509,4337.047740,10035.114189,14967.074371,103.293053,8765.377008,12302.988495
75%,279.867251,365.428638,257.545330,109.720277,20303.638685,279.826139,5486.957595,11886.609768,17496.947296,166.362263,10024.691731,14600.509941
max,863.134577,1076.116013,776.854289,217.592987,38788.084376,738.125263,7445.845759,25752.386429,34924.144012,528.258545,15597.095299,28681.648468


In [ ]:
df_HL_psnr.to_csv('/content/drive/MyDrive/Image_Processing_Plants/PSNR_Healthy_Public.csv',index=False)

In [ ]:
df_HL_ssim.to_csv('/content/drive/MyDrive/Image_Processing_Plants/SSIM_Healthy_Public.csv',index=False)
df_HL_mse.to_csv('/content/drive/MyDrive/Image_Processing_Plants/MSE_Healthy_Public.csv',index=False)

In [ ]:
df_D_psnr.to_csv('/content/drive/MyDrive/Image_Processing_Plants/PSNR_Diseased_Public.csv',index=False)
df_D_ssim.to_csv('/content/drive/MyDrive/Image_Processing_Plants/SSIM_Diseased_Public.csv',index=False)
df_D_mse.to_csv('/content/drive/MyDrive/Image_Processing_Plants/MSE_Diseased_Public.csv',index=False)

In [ ]:
for dir in folders:
  for _,_,files in os.walk(dir):
    for f in files:

# Diseases data set      
      if("B.Spot" in f):
        image = cv2.imread(os.path.join(dir,f))
        # gaussian filter
        gaussian_image = cv2.GaussianBlur(image,(5,5),0)
        
        # unsharp mask filter
        sharp = np.zeros_like(image)
        for i in range(3):
          sharp[:,:,i] = unsharp(image[:,:,i], 5, 0.8)
        
        cv2.imwrite('/content/drive/MyDrive/Image_Processing_Plants/Filtered_Data_Public/Diseased_Public_Gaussian/gaussian_'+f,gaussian_image)
        cv2.imwrite('/content/drive/MyDrive/Image_Processing_Plants/Filtered_Data_Public/Diseased_Public_Unsharp/unsharp_'+f,sharp)

# Healthy Data Set      
      elif("HL" in f): 
        image = cv2.imread(os.path.join(dir,f))
        # gaussian filter
        gaussian_image = cv2.GaussianBlur(image,(5,5),0)
        
        # unsharp mask filter
        sharp = np.zeros_like(image)
        for i in range(3):
          sharp[:,:,i] = unsharp(image[:,:,i], 5, 0.8)
        
        cv2.imwrite('/content/drive/MyDrive/Image_Processing_Plants/Filtered_Data_Public/Healthy_Public_Gaussian/gaussian_'+f,gaussian_image)
        cv2.imwrite('/content/drive/MyDrive/Image_Processing_Plants/Filtered_Data_Public/Healthy_Public_Unsharp/unsharp_'+f,sharp)

